In [ ]:
import pandas as pd
from numpy import *

dataSet = [line.split() for line in open('/content/mushroom.dat').readlines()]

In [ ]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))

In [ ]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData
    

In [ ]:
C1 = createC1(dataSet)

In [ ]:
D = list(map(set,dataSet))

In [ ]:
L1,suppDat0 = scanD(D,C1,0.5)
L1

[frozenset({'53'}),
 frozenset({'24'}),
 frozenset({'39'}),
 frozenset({'2'}),
 frozenset({'90'}),
 frozenset({'86'}),
 frozenset({'85'}),
 frozenset({'76'}),
 frozenset({'67'}),
 frozenset({'63'}),
 frozenset({'59'}),
 frozenset({'36'}),
 frozenset({'34'})]

In [ ]:
def aprioriGen(Lk, k): 
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: 
                retList.append(Lk[i] | Lk[j]) 
    return retList

In [ ]:
def apriori(dataSet, minSupport):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [ ]:
L,suppData = apriori(dataSet, minSupport = 0.5)
L[0]

[frozenset({'53'}),
 frozenset({'24'}),
 frozenset({'39'}),
 frozenset({'2'}),
 frozenset({'90'}),
 frozenset({'86'}),
 frozenset({'85'}),
 frozenset({'76'}),
 frozenset({'67'}),
 frozenset({'63'}),
 frozenset({'59'}),
 frozenset({'36'}),
 frozenset({'34'})]

In [ ]:
aprioriGen(L[0],2)

[frozenset({'24', '53'}),
 frozenset({'39', '53'}),
 frozenset({'2', '53'}),
 frozenset({'53', '90'}),
 frozenset({'53', '86'}),
 frozenset({'53', '85'}),
 frozenset({'53', '76'}),
 frozenset({'53', '67'}),
 frozenset({'53', '63'}),
 frozenset({'53', '59'}),
 frozenset({'36', '53'}),
 frozenset({'34', '53'}),
 frozenset({'24', '39'}),
 frozenset({'2', '24'}),
 frozenset({'24', '90'}),
 frozenset({'24', '86'}),
 frozenset({'24', '85'}),
 frozenset({'24', '76'}),
 frozenset({'24', '67'}),
 frozenset({'24', '63'}),
 frozenset({'24', '59'}),
 frozenset({'24', '36'}),
 frozenset({'24', '34'}),
 frozenset({'2', '39'}),
 frozenset({'39', '90'}),
 frozenset({'39', '86'}),
 frozenset({'39', '85'}),
 frozenset({'39', '76'}),
 frozenset({'39', '67'}),
 frozenset({'39', '63'}),
 frozenset({'39', '59'}),
 frozenset({'36', '39'}),
 frozenset({'34', '39'}),
 frozenset({'2', '90'}),
 frozenset({'2', '86'}),
 frozenset({'2', '85'}),
 frozenset({'2', '76'}),
 frozenset({'2', '67'}),
 frozenset({'2', '63

In [ ]:
def generateRules(L, supportData, minConf):  
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

In [ ]:
def calcConf(freqSet, H, supportData, brl, minConf):
    prunedH = [] 
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] 
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [ ]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): 
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):  
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [ ]:
L,suppData= apriori(dataSet,minSupport=0.5)

In [ ]:
rules= generateRules(L,suppData, minConf=0.8)

frozenset({'24'}) --> frozenset({'34'}) conf: 0.9557708508845831
frozenset({'24'}) --> frozenset({'85'}) conf: 1.0
frozenset({'24'}) --> frozenset({'86'}) conf: 0.95787700084246
frozenset({'24'}) --> frozenset({'90'}) conf: 0.9283909014321822
frozenset({'53'}) --> frozenset({'34'}) conf: 1.0
frozenset({'53'}) --> frozenset({'85'}) conf: 1.0
frozenset({'53'}) --> frozenset({'86'}) conf: 1.0
frozenset({'53'}) --> frozenset({'90'}) conf: 1.0
frozenset({'2'}) --> frozenset({'85'}) conf: 1.0
frozenset({'39'}) --> frozenset({'34'}) conf: 0.9625801853171776
frozenset({'39'}) --> frozenset({'36'}) conf: 0.811831789023521
frozenset({'39'}) --> frozenset({'85'}) conf: 1.0
frozenset({'39'}) --> frozenset({'86'}) conf: 0.9657875980042766
frozenset({'39'}) --> frozenset({'90'}) conf: 0.8866714183891662
frozenset({'36'}) --> frozenset({'34'}) conf: 0.9691720493247211
frozenset({'34'}) --> frozenset({'36'}) conf: 0.834217841799343
frozenset({'59'}) --> frozenset({'34'}) conf: 0.9629057187017002
froze